In [289]:
import pandas as pd
import numpy as np

from sklearn import preprocessing, svm, model_selection
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score
#from sklearn.model_selection import train_test_split

# Reads in Data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [290]:
# Creates SalesPrice Column
#houses['SalePrice'] = 0.00

# Selects Most Meaningful Columns to work with
houses = houses[["Id", "SalePrice", "LotArea", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "GrLivArea", "BsmtFullBath", "BsmtHalfBath", "FullBath", "HalfBath", "BedroomAbvGr", "YrSold", "YearBuilt", "OverallQual", "OverallCond"]]
#print(houses.head())

KeyError: "['TotalBsmtSF' '1stFlrSF' '2ndFlrSF' 'GrLivArea' 'BsmtFullBath'\n 'BsmtHalfBath' 'FullBath' 'HalfBath' 'YrSold' 'YearBuilt'] not in index"

In [287]:
# Creates new features: TotSqf, TotSqf2, Age, Age2, TotFullBath, TotHalfBath

# Square Footage 
houses["TotSqf"] = (houses['TotalBsmtSF'] + houses["1stFlrSF"] + houses["2ndFlrSF"])
houses["TotSqf2"] = houses["TotSqf"] * houses["TotSqf"]

# Age 
houses["Age"] = houses["YrSold"] - houses["YearBuilt"]
houses["Age2"] = houses["Age"] * houses["Age"]

# Num Full/Half Baths
houses["TotFullBath"] = houses["BsmtFullBath"] + houses["FullBath"]
houses["TotHalfBath"] = houses["BsmtHalfBath"] + houses["HalfBath"]

houses.head()

KeyError: 'TotalBsmtSF'

In [283]:
# Select only those columns that we will need for our regression and sample submission
houses = houses[["Id", "SalePrice", "LotArea", "TotSqf", "TotSqf2", "Age", "Age2", "TotFullBath", "TotHalfBath", "BedroomAbvGr", "OverallQual", "OverallCond"]]
print(houses.head())

   Id  SalePrice  LotArea  TotSqf   TotSqf2  Age  Age2  TotFullBath  \
0   1     208500     8450    2566   6584356    5    25            3   
1   2     181500     9600    2524   6370576   31   961            2   
2   3     223500    11250    2706   7322436    7    49            3   
3   4     140000     9550    2473   6115729   91  8281            2   
4   5     250000    14260    3343  11175649    8    64            3   

   TotHalfBath  BedroomAbvGr  OverallQual  OverallCond  
0            1             3            7            5  
1            1             3            6            8  
2            1             3            7            5  
3            0             3            7            5  
4            1             4            8            5  


In [284]:
# # Sets forecast column to a string to make it easy to change if you want to repurpose this code for later.
# forecast_col = "SalePrice"
# # Fills in NA values - I don't think we have any with the variables currently selected but it's a good habit.
# houses.fillna(-99999, inplace=True)

# # Designate "predictors" - explanatory variables
# #predictors = houses.columns.values[1:11]
# #print(predictors)

# X = np.array(houses.drop(["SalePrice"],1))
# #print(X)

# y = np.array(houses["SalePrice"])
# #print(y)

# X = preprocessing.scale(X)
# #print(X)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# classifier = LinearRegression()
# classifier.fit(X_train, y_train)
# accuracy = classifier.score(X_test, y_test)

# print(accuracy)

# #print(houses["SalesPrice"])

In [278]:
#predictors = ["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]
# alg = LinearRegression()
# kf = KFold(titanic.shape[0], n_folds=3, random_state=1)
# predictions = []
# for train, test in kf:
#     train_predictors = (titanic[predictors].iloc[train,:])
#     train_target = titanic["Survived"].iloc[train]
#     alg.fit(train_predictors, train_target)
#     test_predictions = alg.predict(titanic[predictors].iloc[test,:])
#     predictions.append(test_predictions)
# predictions = np.concatenate(predictions, axis=0)

print(houses.head())
predictors = ["LotArea", "TotSqf", "TotSqf2", "Age", "Age2", "TotFullBath", "TotHalfBath", "BedroomAbvGr", "OverallQual", "OverallCond"]
alg = LinearRegression()
kf = KFold(10, shuffle=True)

print(kf.get_n_splits(X))
print(kf)
predictions = []

# for train, test in kf.split(X):
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]
# TRAIN: [2 3] TEST: [0 1]
# TRAIN: [0 1] TEST: [2 3]


for train, test in kf.split(X):
    train_predictors = (houses[predictors].iloc[train,:])
    train_target = houses["SalePrice"].iloc[train]
    alg.fit(train_predictors, train_target)
    test_predictions = alg.predict(houses[predictors].iloc[test,:])
    predictions.append(test_predictions)
predictions = np.concatenate(predictions, axis=0)

print(predictions)

houses["SalePrice"] = predictions

print(houses.head())    

       Id  SalePrice  LotArea  TotSqf    TotSqf2   Age    Age2  TotFullBath  \
0  1461.0     208500  11622.0  1778.0  3161284.0  49.0  2401.0          1.0   
1  1462.0     181500  14267.0  2658.0  7064964.0  52.0  2704.0          1.0   
2  1463.0     223500  13830.0  2557.0  6538249.0  13.0   169.0          2.0   
3  1464.0     140000   9978.0  2530.0  6400900.0  12.0   144.0          2.0   
4  1465.0     250000   5005.0  2560.0  6553600.0  18.0   324.0          2.0   

   TotHalfBath  BedroomAbvGr  OverallQual  OverallCond  
0          0.0           2.0          5.0          6.0  
1          1.0           3.0          6.0          6.0  
2          1.0           3.0          5.0          5.0  
3          1.0           3.0          6.0          6.0  
4          0.0           2.0          8.0          5.0  
10
KFold(n_splits=10, random_state=None, shuffle=True)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [279]:
for column in houses:
    if column != "SalePrice" and column != "Id":
        houses.drop(column, 1, inplace=True)
houses.to_csv("final.csv", index=False, float_format='%.0f')